# Exercise 7.01

In [1]:
# Import the Libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from tensorflow import random

In [2]:
#set a seed
seed = 1
np.random.seed(seed)
random.set_seed(seed)

# Initialising the CNN
classifier = Sequential()

Add convolutional layers

In [3]:
# Convolution
classifier.add(Conv2D(filters = 32, kernel_size = (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Conv2D Output Size = (64 - 3 + 0)/1 + 1 = 62 --> (62, 62) * 32

# Pooling
classifier.add(MaxPool2D(pool_size = (2, 2)))
# MaxPool2D Output Size = (62 - 2 + 0)/2 + 1 = 31 --> (31, 31) * 32

#  Flattening
classifier.add(Flatten())
# Flattening Output Size = 31 * 31 * 32 = 30752

Add dense layers

In [4]:
# Full ANN Connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [5]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 30752)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               3936384   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 3,937,409
Trainable params: 3,937,409
Non-trainable params: 0
_________________________________________________________________


Add data generators

In [6]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

Create training and test datasets

In [7]:
# here we can change the batch_size. 

training_set = train_datagen.flow_from_directory('../dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('../dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 10764 images belonging to 2 classes.
Found 2674 images belonging to 2 classes.


Fit the model

In [8]:
# Here we test different fittings!

# steps_per_epoch = AllData number / batch_size 

# result --> `fit_generator` is deprecated and will be removed in a future version.
#1# classifier.fit_generator(training_set, steps_per_epoch = 10000, epochs = 2, validation_data = test_set, validation_steps = 2500, shuffle=False)

# result --> Error: Your input ran out of data
#2# classifier.fit(training_set, steps_per_epoch = 10000, epochs = 2, validation_data = test_set, validation_steps = 2500, shuffle=False)


# steps_per_epoch = 10764 / 32 = 336
# validation_steps = 2674 / 32 = 83
#3# 

classifier.fit(training_set, steps_per_epoch = 336, epochs = 2, validation_data = test_set, validation_steps = 83, shuffle=False)

Epoch 1/2
336/336 [==============================] - 23s 68ms/step - loss: 0.4905 - accuracy: 0.7727 - val_loss: 0.4490 - val_accuracy: 0.8012
Epoch 2/2
336/336 [==============================] - 24s 70ms/step - loss: 0.4028 - accuracy: 0.8223 - val_loss: 0.3893 - val_accuracy: 0.8223
